# Inferencing the Language Model
This takes the input from a text file and get the results exported in a csv file.

In [1]:
from __future__ import print_function

from keras.models import Model, load_model
from keras.layers import Input
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 5000  # Number of samples to train on.
# Path to the data txt file on disk.
#data_path = 'fra-eng/fra.txt'
data_path = 'path/to/datset.csv'

import os
#User all GPU's
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.


# Tokenize the input and output text in individual words

In [2]:
# Vectorize the data.  We use the same approach as the training script.
# NOTE: the data must be identical, in order for the character -> integer
# mappings to be consistent.
# We omit encoding target_texts since they are not needed.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    try:
        input_text, target_text = line.split(',')
    except: #code to run if error occurs
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
        #print("An exception occurred ",target_text)
        pass
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    #target_text = '\t' + target_text.split()[0] + '\n'
    #target_text = '\t' + target_text + '\n'
    
    for i in range(0,6):
        try:
        
            input_text_m = input_text.split()[i]+ ' '
            input_texts.append(input_text_m)
            target_text_m = '\t' +  target_text.split()[i] + ' ' + '\n'
            target_texts.append(target_text_m)
            
            for char in input_text_m:
                if char not in input_characters:
                    input_characters.add(char)
            for char in target_text_m:
                if char not in target_characters:
                    target_characters.add(char)
        except:
            pass
        
print (input_texts)
print (target_texts)


['भिमराव ', 'वेणुनाथ ', 'आगळे ', 'हरिचंद्र ', 'वेणुनाथ ', 'आगळे ', 'शिवनारायण ', 'वेणुनाथ ', 'आगळे ', 'लालासाहेब ', 'रामचंद्र ', 'आगळे ', 'सतिष ', 'रामचंद्र ', 'आगळे ', 'दशरथ ', 'बंडा ', 'आगळे ', 'दत्तात्रय ', 'शंकरराव ', 'बरकते ', 'राजाभाऊ ', 'शंकरराव ', 'बरकते ', 'लालासाहेब ', 'दत्तात्रय ', 'बरकते ', 'गणेश ', 'राजाभाऊ ', 'बरकते ', 'लालासाहेब ', 'दत्तात्रय ', 'बरकते ', 'भारत ', 'काशिनाथ ', 'आगळे ', 'पार्वतीबाई ', 'महादेव ', 'आगळे ', 'महेश ', 'श्रीराम ', 'आगळे ', 'राजाभाऊ ', 'शंकरराव ', 'बरकते ', 'बालाजी ', 'धोंडीराम ', 'आगळे ', 'बद्रीनारायण ', 'हरीश्चंद्र ', 'आगळे ', 'बजरंग ', 'हरीश्चंद्र ', 'आगळे ', 'शिवनारायण ', 'वेणुनाथ ', 'आगळे ', 'शिवनारायण ', 'वेणुनाथ ', 'आगळे ', 'दादासाहेब ', 'भगवान ', 'दसवंते ', 'बालासाहेब ', 'बाबुराव ', 'आगळे ', 'शंकुतला ', 'जीवन ', 'दसवंते ', 'विश्वनाथ ', 'किसन ', 'दसवंते ', 'धोंडीराम ', 'हरीभाऊ ', 'आगळे ', 'अर्चना ', 'मारुती ', 'अंधारे ', 'स्वप्निल ', 'मारुती ', 'अंधारे ', 'ओमप्रकाश ', 'चत्रभुज ', 'आगळे ', 'बळीराम ', 'चत्रभुज ', 'आगळे ', 'उषा ', 'लहुदास ', 

# Identify the number of input and output tokens at character level
These token are fetched out based on the input data provided whcih is less than we actually have in language, if we have exceesive data provided then it should all the chaacyters of correspoding script

In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens,input_characters)
print('Number of unique output tokens:', num_decoder_tokens,target_characters)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 15509
Number of unique input tokens: 75 [' ', '(', ')', '-', '.', '/', '0', '1', '2', '3', '4', '6', 'ँ', 'ं', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ऱ', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'क़', '३', '\u200d']
Number of unique output tokens: 42 ['\t', '\n', ' ', '"', '(', ')', '-', '.', '/', '0', '1', '2', '3', '4', '6', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'y', 'ऑ', 'ऱ', 'ॅ', 'ॉ', 'क़', '\u200d']
Max sequence length for inputs: 23
Max sequence length for outputs: 30


# Indexing the characters
Here we are indexing all the characters at source as well as target text and creating a placeholders for 
encoder_input_data, decoder_input_data and decoder_target_data matrices


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

for i, input_text in enumerate(input_texts):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

print(input_texts)


['भिमराव ', 'वेणुनाथ ', 'आगळे ', 'हरिचंद्र ', 'वेणुनाथ ', 'आगळे ', 'शिवनारायण ', 'वेणुनाथ ', 'आगळे ', 'लालासाहेब ', 'रामचंद्र ', 'आगळे ', 'सतिष ', 'रामचंद्र ', 'आगळे ', 'दशरथ ', 'बंडा ', 'आगळे ', 'दत्तात्रय ', 'शंकरराव ', 'बरकते ', 'राजाभाऊ ', 'शंकरराव ', 'बरकते ', 'लालासाहेब ', 'दत्तात्रय ', 'बरकते ', 'गणेश ', 'राजाभाऊ ', 'बरकते ', 'लालासाहेब ', 'दत्तात्रय ', 'बरकते ', 'भारत ', 'काशिनाथ ', 'आगळे ', 'पार्वतीबाई ', 'महादेव ', 'आगळे ', 'महेश ', 'श्रीराम ', 'आगळे ', 'राजाभाऊ ', 'शंकरराव ', 'बरकते ', 'बालाजी ', 'धोंडीराम ', 'आगळे ', 'बद्रीनारायण ', 'हरीश्चंद्र ', 'आगळे ', 'बजरंग ', 'हरीश्चंद्र ', 'आगळे ', 'शिवनारायण ', 'वेणुनाथ ', 'आगळे ', 'शिवनारायण ', 'वेणुनाथ ', 'आगळे ', 'दादासाहेब ', 'भगवान ', 'दसवंते ', 'बालासाहेब ', 'बाबुराव ', 'आगळे ', 'शंकुतला ', 'जीवन ', 'दसवंते ', 'विश्वनाथ ', 'किसन ', 'दसवंते ', 'धोंडीराम ', 'हरीभाऊ ', 'आगळे ', 'अर्चना ', 'मारुती ', 'अंधारे ', 'स्वप्निल ', 'मारुती ', 'अंधारे ', 'ओमप्रकाश ', 'चत्रभुज ', 'आगळे ', 'बळीराम ', 'चत्रभुज ', 'आगळे ', 'उषा ', 'लहुदास ', 

# Load saved model for inferencing

In [5]:
# Restore the model and construct the encoder and decoder.
model = load_model('s2shin.h5')

W1023 17:07:15.518077 18332 deprecation.py:323] From C:\Users\moharsha\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1023 17:07:16.452576 18332 deprecation_wrapper.py:119] From C:\Users\moharsha\.conda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



# Print layers in model
Just for debugging in case layers conflicts

In [6]:
layer_name=[]

for layer in model.layers:
    print(layer.name)
    layer_name.append(layer.name)

input_1
input_2
lstm_1
lstm_2
dense_1


# Define sampling models

In [7]:
encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_5')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.

In [8]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

# Decodes an input sequence.

In [9]:
# Decodes an input sequence.
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [10]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]

    decoded_sentence = decode_sequence(input_seq)

# Pass the input text for inferencing
We are redoing this step; first we load the data and tensor as per the training now we are creating for 
Inferencing

In [11]:
# Vectorize the data.  We use the same approach as the training script.
# NOTE: the data must be identical, in order for the character -> integer
# mappings to be consistent.
# We omit encoding target_texts since they are not needed.

#input_texts = []
#target_texts = []
input_characters = set()
target_characters = set()
outputNames =[]
outputNamesList =[]
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(1000, len(lines) - 1)]:
    input_texts = []
    target_texts = []
    input_text_print = ''
    target_texts_print = ''
    decoded_text_print = ''
    outputNames =[]

    
    try:
        input_text, target_text = line.split(',')
    except: #code to run if error occurs
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
        #print("An exception occurred ",target_text)
        pass
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    #target_text = '\t' + target_text.split()[0] + '\n'
    #target_text = '\t' + target_text + '\n'
    

  
    for i in range(0,len(input_text.split())):
        try:
            input_text_m = input_text.split()[i]+ ' '
            input_texts.append(input_text_m)
            target_text_m = '\t' +  target_text.split()[i] + ' ' + '\n'
            target_texts.append(target_text_m)
            #print(input_texts)

        except:
            print('pass')
            pass
        
    encoder_input_data = np.zeros(
            (len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')    
    for j, input_text in enumerate(input_texts):
        #print(input_text)
       
        for t, char in enumerate(input_text):
            #print(j,t,char,input_token_index[char])
            try:
                encoder_input_data[j, t, input_token_index[char]] = 1.
            except:
                pass

    for seq_index in range(len(input_texts)):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = encoder_input_data[seq_index: seq_index + 1]

        decoded_sentence = decode_sequence(input_seq)
        input_text_print = input_text_print +''+input_texts[seq_index]
        target_texts_print = target_texts_print + ' '+target_texts[seq_index].split()[0]
        decoded_text_print = decoded_text_print.rstrip() + ' '+decoded_sentence.rstrip()
    
    outputNames.append(input_text_print)
    outputNames.append(target_texts_print)
    outputNames.append(decoded_text_print)
    outputNamesList.append(outputNames)
    
print(outputNamesList)

[['भिमराव वेणुनाथ आगळे ', ' bhimarava venunatha agale', ' bhimarava venunatha agale'], ['हरिचंद्र वेणुनाथ आगळे ', ' harichandra venunatha agale', ' harichandra venunatha agale'], ['शिवनारायण वेणुनाथ आगळे ', ' shivanarayana venunatha agale', ' shivanarayana venunatha agale'], ['लालासाहेब रामचंद्र आगळे ', ' lalasaheba ramachandra agale', ' sopana ramachandra agale'], ['सतिष रामचंद्र आगळे ', ' satisha ramachandra agale', ' sativa ramachandra agale'], ['दशरथ बंडा आगळे ', ' dasharatha banda agale', ' dasharatha dangade agale'], ['दत्तात्रय शंकरराव बरकते ', ' dattatraya shankararava barakate', ' dattatraya shankararava barakate'], ['राजाभाऊ शंकरराव बरकते ', ' rajabhau shankararava barakate', ' ramabhau shankararava barakate'], ['लालासाहेब दत्तात्रय बरकते ', ' lalasaheba dattatraya barakate', ' sopana dattatraya barakate'], ['गणेश राजाभाऊ बरकते ', ' ganesha rajabhau barakate', ' ganesha ramabhau barakate'], ['लालासाहेब दत्तात्रय बरकते ', ' lalasaheba dattatraya barakate', ' sopana dattatraya 

#  Export the Output

In [12]:
import csv
# name of csv file 
filename = "TranscriptedNames.csv"
# writing to csv file 
with open(filename, 'w',encoding='utf-8') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
             
    # writing the data rows 
    csvwriter.writerows(outputNamesList)